In [85]:
import numpy as np
import csv
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold
from scipy import interp
from  matplotlib import pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

In [21]:
class passengerManager():

    featureName = ['Pclass', 'Name', \
                    'Sex', 'Age', \
                    'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
    survivals = {"Survived": 1, "Not Survived" : 0}
    m_path = 'pkl/'

In [23]:
    def __init__(self):
            self.model = None
            self.features = []
            self.normalizedFeatures = []
            self.labels = []
            self.n_classes = 0
            self.paramPath = ""


    def initializeTrainData(self, data):
        self.features, self.normalizedFeatures = self.parseTrainFeatures(data)
        self.labels = list(map(lambda x: passengerManager.survivals[x.survival], data))
        self.n_classes = len(np.unique(self.labels))


    def calculateAve(self, feature):
        return [np.mean(a) for a in feature]


    def calculateStd(self, feature):
        return [np.std(a) for a in feature]

In [3]:
with open("train.csv", 'r') as csvfile:
        matrixReader = csv.reader(csvfile, delimiter= ',', quotechar='|')


In [102]:
data = pd.read_csv('train.csv')

In [103]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,NaN,C


In [104]:
data.drop(data.columns[[0, 3]], axis=1, inplace=True)

In [105]:
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35,1,0,113803,53.1000,C123,S
4,0,3,male,35,0,0,373450,8.0500,NaN,S
5,0,3,male,NaN,0,0,330877,8.4583,NaN,Q
6,0,1,male,54,0,0,17463,51.8625,E46,S
7,0,3,male,2,3,1,349909,21.0750,NaN,S
8,1,3,female,27,0,2,347742,11.1333,NaN,S
9,1,2,female,14,1,0,237736,30.0708,NaN,C


In [106]:
data['Sex'] = data['Sex'].map({'female': 1, 'male': 0})

In [107]:
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,0,22,1,0,A/5 21171,7.2500,NaN,S
1,1,1,1,38,1,0,PC 17599,71.2833,C85,C
2,1,3,1,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,1,35,1,0,113803,53.1000,C123,S
4,0,3,0,35,0,0,373450,8.0500,NaN,S
5,0,3,0,NaN,0,0,330877,8.4583,NaN,Q
6,0,1,0,54,0,0,17463,51.8625,E46,S
7,0,3,0,2,3,1,349909,21.0750,NaN,S
8,1,3,1,27,0,2,347742,11.1333,NaN,S
9,1,2,1,14,1,0,237736,30.0708,NaN,C


In [100]:
print (data.describe())

         Survived      Pclass         Sex         Age       SibSp       Parch  \
count  891.000000  891.000000  891.000000  714.000000  891.000000  891.000000   
mean     0.383838    2.308642    0.352413   29.699118    0.523008    0.381594   
std      0.486592    0.836071    0.477990   14.526497    1.102743    0.806057   
min      0.000000    1.000000    0.000000    0.420000    0.000000    0.000000   
25%      0.000000    2.000000    0.000000   20.125000    0.000000    0.000000   
50%      0.000000    3.000000    0.000000   28.000000    0.000000    0.000000   
75%      1.000000    3.000000    1.000000   38.000000    1.000000    0.000000   
max      1.000000    3.000000    1.000000   80.000000    8.000000    6.000000   

             Fare  
count  891.000000  
mean    32.204208  
std     49.693429  
min      0.000000  
25%      7.910400  
50%     14.454200  
75%     31.000000  
max    512.329200  


In [108]:
y = data["Survived"].as_matrix()

In [111]:
features.drop(features.columns[[0]], axis=1, inplace=True)

In [112]:
features

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
2,3,1,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,35,1,0,113803,53.1000,C123,S
4,3,0,35,0,0,373450,8.0500,NaN,S
5,3,0,NaN,0,0,330877,8.4583,NaN,Q
6,1,0,54,0,0,17463,51.8625,E46,S
7,3,0,2,3,1,349909,21.0750,NaN,S
8,3,1,27,0,2,347742,11.1333,NaN,S
9,2,1,14,1,0,237736,30.0708,NaN,C
10,3,1,4,1,1,PP 9549,16.7000,G6,S
11,1,1,58,0,0,113783,26.5500,C103,S


In [113]:
features['Ticket_enc'] = pd.factorize(features['Ticket'])[0]
features['Cabin_enc'] = pd.factorize(features['Cabin'])[0]
features['Embarked_enc'] = pd.factorize(features['Embarked'])[0]

In [115]:
features.drop(features.columns[[5, 7, 8]], axis=1, inplace=True)

In [116]:
features

,Pclass,Sex,Age,SibSp,Parch,Ticket_enc
2,3,1,26,0,0,0
3,1,1,35,1,0,1
4,3,0,35,0,0,2
5,3,0,NaN,0,0,3
6,1,0,54,0,0,4
7,3,0,2,3,1,5
8,3,1,27,0,2,6
9,2,1,14,1,0,7
10,3,1,4,1,1,8
11,1,1,58,0,0,9


In [118]:
features['Age'].fillna(features['Age'].mean())

2      26.000000
3      35.000000
4      35.000000
5      29.698272
6      54.000000
7       2.000000
8      27.000000
9      14.000000
10      4.000000
11     58.000000
12     20.000000
13     39.000000
14     14.000000
15     55.000000
16      2.000000
17     29.698272
18     31.000000
19     29.698272
20     35.000000
21     34.000000
22     15.000000
23     28.000000
24      8.000000
25     38.000000
26     29.698272
27     19.000000
28     29.698272
29     29.698272
30     40.000000
31     29.698272
         ...    
861    21.000000
862    48.000000
863    29.698272
864    24.000000
865    42.000000
866    27.000000
867    31.000000
868    29.698272
869     4.000000
870    26.000000
871    47.000000
872    33.000000
873    47.000000
874    28.000000
875    15.000000
876    20.000000
877    19.000000
878    29.698272
879    56.000000
880    25.000000
881    33.000000
882    22.000000
883    28.000000
884    25.000000
885    39.000000
886    27.000000
887    19.000000
888    29.6982

In [126]:
features.describe()

,Pclass,Sex,Age,SibSp,Parch,Ticket_enc
count,889.000000,889.000000,712.000000,889.000000,889.000000,889.000000
mean,2.309336,0.352081,29.698272,0.521935,0.382452,305.548931
std,0.835537,0.477888,14.540716,1.103752,0.806761,196.971016
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,20.000000,0.000000,0.000000,134.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,284.000000
75%,3.000000,1.000000,38.000000,1.000000,0.000000,473.000000
max,3.000000,1.000000,80.000000,8.000000,6.000000,678.000000


In [119]:
x = features.as_matrix()

In [122]:
y = y[0:889]

In [31]:
x[0:2]

array([[3, 1, 26.0, 0, 0, 'STON/O2. 3101282', 7.925, nan, 'S'],
       [1, 1, 35.0, 1, 0, '113803', 53.1, 'C123', 'S']], dtype=object)

In [123]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

In [124]:
lr = LogisticRegression()

In [125]:
lr.fit(x_train,y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').